## Classifier patterns

Matthew Stone, CS 533, Spring 2017, to accompany first homework.

This notebook is designed to get you started thinking and coding for the first homework.  It includes illustrations of how to use the software that I have provided for the assignment.  It includes some cool bits of python that you might not have thought of (or known to google for) that make many of the things you'll have to do very easy.  And it includes [some design patterns][1] for exploring representation changes in classification problems that you can work with as you push your ideas forward.

[1]:https://en.wikipedia.org/wiki/Software_design_pattern

In [1]:
import os
import nltk
import re
import itertools
import vocabulary
import newsreader
import numpy as np
import scipy
import sklearn
try:
    import cPickle as pickle
except:
    import pickle

### Design Pattern 0: Keep the things you have to customize separate

It's always easy to get carried away getting settled into a particular setup and working on a particular experiment.  But the first step in making your work general is [DRY: don't repeat yourself][1].  That means setting up variables once and for all for the things you're going to have to get right to work with particular data in a particular environment.  Let's get organized.

[1]:https://en.wikipedia.org/wiki/Don't_repeat_yourself

In [2]:
vocab_file, vocab_file_type = "vocab.pkl", "pickle"

embedding_file, embedding_dimensions, embedding_cache = \
    "glove.6B.50d.txt", 50, "embedding.npz"

train_dir, test_dir = \
    "20news-bydate/20news-bydate-train/", "20news-bydate/20news-bydate-test/"

AUTOS, MOTORCYCLES, RELIGION = \
    "rec.autos", "rec.motorcycles", "talk.religion.misc"

### Design Pattern 1: Save your work

You won't always be able to keep your notebook running indefinitely.  Some of the computations you will want to do in putting together the assignment are big.  So consider using the [python pickle serialization framework][1] and [loading and saving functionality from numpy][2] in cases like this to let you pick up right where you left off.

[1]:http://www.ibm.com/developerworks/library/l-pypers/index.html
[2]:https://docs.scipy.org/doc/numpy/reference/generated/numpy.load.html

In [3]:
made_vocabulary = True
if made_vocabulary :
    v = vocabulary.Vocabulary.load(vocab_file, file_type=vocab_file_type)
else: 
    v = vocabulary.Vocabulary.from_iterable(newsreader.all_20newsgroup_tokens("20news-bydate"),
                                            file_type=vocab_file_type)
    v.save(vocab_file)
v.stop_growth()

In [4]:
made_embedding = True
if made_embedding :
    e = newsreader.load_sparse_csr(embedding_cache)
else: 
    e = newsreader.build_sparse_embedding(v, embedding_file, embedding_dimensions)
    newsreader.save_sparse_csr(embedding_cache, e)

### Example 1: Finding features

You're going to want to use the data manager class, which has placeholders that say how you might build a feature dictionary by using the training data and then how you will scan files to create feature matrices from file tokens.  Here are default functions that carry out those operations in the simplest way possible.  **Remember: simple is good.** Being fancy is just as likely to get you into trouble as it is to help you.  If you have these functions defined, you can make a data manager with a call like this (where `v` is your vocabulary)

```python
my_data = newsreader.DataManager(class1_training_directory,
                                 class2_training_directory,
                                 class1_test_directory,
                                 class2_test_directory,
                                 use_default_features(v), 
                                 count_features)
```
                                 

In [5]:
def use_default_features(vocab) :
    return lambda data: v

def count_features(features, gen_tokens) :
    for t in gen_tokens :
        r = features.add(t)
        if r :
            yield r    

### Design Pattern 2: Higher-order functions for one-off abstraction

We're going to be using `scipy` sparse matrices to handle language data (as we must), but unfortunately it turns out that a lot of the nice operations that you can do on `numpy` matrices don't work on sparse matrices, because they wouldn't preserve the sparsity in the general case.  One of the things that turns out to be impossible is to "forget" the  values in a matrix and replace all the nonzero values in a matrix with a sensible value like 1.  Bottom line: if we want to have yes/no features in our data matrices, rather than count features, we're going to have to remove redundancies right away when we scan the file.

Now, if you think for a second, you'll realize that this effect doesn't have anything to do with what features we're looking for or how we're looking for them.  Features are always integers, and the way we merge redundancies is to accumulate the features that we've seen so far into a set and, once we're done scanning a file, we then emit the features we observed.

Lots of languages (but maybe not the ones that you're used to) let you just parameterize this operation by the basic `feature_counter` operation, and create abstract, general code without the nuisance of making objects, classes, data types or other heavy-duty discipline.  Here's how you do it in python.  As it happens, I wanted to experiment with binary features in exploring the design space for text classifiers today, so this is what I will use.

In [6]:
def make_binary_features(feature_counter) :
    def collect_features(features, gen_tokens) :
        seen = set()
        for f in feature_counter(features, gen_tokens) :
            seen.add(f)
        for f in seen :
            yield f
    return collect_features

autos_vs_religion = newsreader.DataManager(train_dir + AUTOS,
                                           train_dir + RELIGION,
                                           test_dir + AUTOS,
                                           test_dir + RELIGION,
                                           use_default_features(v),
                                           make_binary_features(count_features))
autos_vs_religion.initialize()

### Design Pattern 3: Bundle your data and provide consistent views

From data management, we've learned that analysts are often interested in [views][1], which are systematically related to underlying data but record the results of inferences and other computations which are precomputed so that you can have fast access to them.

This is going to be important as you explore the best ways  to represent the underlying text, and design experiments that are going to compare these representations to one another.  

This is the kind of thing that it's nice to formalize with a python class.  The `Experiment` class below sets up the problem of learning a classifier from a data manager, arranges the data acquisition and classifier training, and carries out a preliminary validation.  Most importantly, it provides a general method for taking your data, rerepresenting it in a consistent way, and setting up another experiment as a result.

### Design Pattern 4: But keep the test data sacred

**You don't want to make it easy to peek at the test data, which is special and limited.** So even though this class keeps the structure in place so that you manage a consistent representation of the test data, you have to add code to evaluate on the test data yourself.


[1]:https://en.wikipedia.org/wiki/View_(SQL)

In [7]:
class Experiment(object) :
    '''Organize the process of getting data, building a classifier,
    and exploring new representations'''
    
    def __init__(self, data, classifier) :
        'set up the problem of learning a classifier from a data manager'
        self.data = data
        self.classifier = classifier
        self.initialized = False
        
    def initialize(self) :
        'materialize the training data, dev data and test data as matrices'
        if not self.initialized :
            self.train_X, self.train_y = self.data.training_data()
            self.dev_X, self.dev_y = self.data.dev_data()
            self.test_X, self.test_y = self.data.test_data()
            self.initialized = True
        
    def fit_and_validate(self) :
        'train the classifier and assess predictions on dev data'
        if not self.initialized :
            self.initialize()
        self.classifier.fit(self.train_X, self.train_y)
        self.dev_predictions = self.classifier.predict(self.dev_X)
        self.accuracy = sklearn.metrics.accuracy_score(self.dev_y, self.dev_predictions)
                
    @classmethod
    def transform(cls, expt, operation, classifier) :
        'use operation to transform the data from expt and set up new classifier'
        if not expt.initialized :
            expt.initialize()
        result = cls(expt.data, classifier)
        result.train_X, result.train_y = operation(expt.train_X, expt.train_y)
        result.dev_X, result.dev_y = operation(expt.dev_X, expt.dev_y)
        result.test_X, result.test_y = operation(expt.test_X, expt.test_y)
        result.initialized = True
        return result
        

### Example 2: Setting up an experiment

Pretty straightforward.

In [8]:
expt1 = Experiment(autos_vs_religion,
                   sklearn.linear_model.SGDClassifier(loss="log",
                                       penalty="elasticnet",
                                       n_iter=5))
expt1.initialize()
expt1.fit_and_validate()
expt1.accuracy

0.93434343434343436

### Example 3: Getting similarity data

One thing that you have to do as part of the assignment is to come up with a way of measuring how different the documents in different classes are. Getting the data is surprisingly easy with a little savvy with `numpy`, `scipy` and `scikit-learn`.  Let's look at the ingredients.

* [scikit-learn's pairwise distances operation.][1]  This is pretty amazing and necessary in that it works on sparse matrices and gives you pairwise distance measures across a set of items simultaneously.  Cosine distance is one of the options.  Passing in one matrix gives you the distances between all the rows in the matrix; passing in two matrices gives you the distances between each row in one and each row in the other.
* [Boolean indexing and row slices][2].  It's absurdly easy to get all the rows of the training data from one class or the other class with this elegant (and surprisingly readable) program construct.  This happens to work nicely here because we've stored the data as a column-sparse-row matrix which allows fast and flexible row slices.

Bottom line: this teeny function gives you a matrix of the distances between items of differing categories, and two matrices of distances between items of the same category, for your further exploration.

[1]:http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_distances.html
[2]:https://docs.scipy.org/doc/numpy-1.10.1/user/basics.indexing.html

In [9]:
def difficulty(expt) :
    # this was too long to write out over and over
    # plus you want to be sure to have the same distance across the board
    # DRY!
    d = lambda *args: sklearn.metrics.pairwise.pairwise_distances(*args, metric='cosine')
    
    Xyes = expt.train_X[expt.train_y ==1, :]
    Xno = expt.train_X[expt.train_y != 1, :] 
    return d(Xyes, Xno), d(Xyes), d(Xno)

### Design Pattern 5: Test

If you want to understand similarity, why not play with what you can compute from the difficulty matrices AUTOS-RELIGION and AUTOS-MOTORCYCLES.  Is the answer what you expect... once you measure distance the right way?

### Design Pattern 6: But remember that your tests ruin the data

Just don't include the AUTOS-RELIGION and AUTOS-MOTORCYCLES comparisons in the final evaluation you put together.  If you've already peeked, and in fact tuned what you've done, your experiment won't tell you anything new!

In [10]:
autos_vs_bikes = newsreader.DataManager(train_dir + AUTOS,
                                        train_dir + MOTORCYCLES,
                                        test_dir+ AUTOS,
                                        test_dir + MOTORCYCLES,
                                        use_default_features(v),
                                        make_binary_features(count_features))
autos_vs_bikes.initialize()

expt1b = Experiment(autos_vs_bikes,
                   sklearn.linear_model.SGDClassifier(loss="log",
                                       penalty="elasticnet",
                                       n_iter=5))
expt1b.initialize()
expt1b.fit_and_validate()
expt1b.accuracy

0.93939393939393945

### Example 4: Making feature values meaningful.

In preparing this assignment, I read [an ACL 2012 paper by Sida Wang and Chris Manning][2] about text classification.  They suggested weighting word features not based on counts, or 1-0, or based on TF-IDF, but instead based on the evidence that the features provide for the eventual class.  This is a cool and influential idea.  All you need to implement it is to work with binary features, compute a score for each feature, and then reweight by the scores.  It's crazy simple code.
* Use boolean indexing and slices to get the data
* Sparse matrices have built-in methods for counting nonzero elements
* Numpy lets you compute the relevant operations elementwise over the matrix.

This is not just a one-off pattern for this particular research result.  For example, if you don't bother to separate the positive and negative examples, and add a call to [np.reciprocal][1], you'll basically turn this into TFIDF weighting.

[1]:https://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.reciprocal.html
[2]:http://aclweb.org/anthology/P/P12/P12-2018.pdf

In [11]:
def reweight_features(expt) :
    Xyes = expt.train_X[expt.train_y ==1, :]
    Xno = expt.train_X[expt.train_y != 1, :] 
    yesrates = np.log((Xyes.getnnz(axis=0) + 1.) / Xyes.shape[1])
    norates = np.log((Xno.getnnz(axis=0) + 1.) / Xno.shape[1])
    W = scipy.sparse.diags(yesrates - norates, 0)
    return lambda X, y: (X.dot(W), y)

The infrastructure that we have in place already to work with data makes this super-easy to try out.

In [12]:
expt2 = Experiment.transform(expt1,
                             reweight_features(expt1),
                            sklearn.linear_model.SGDClassifier(loss="log",
                                       penalty="elasticnet",
                                       n_iter=5))
expt2.fit_and_validate()

expt2.accuracy

0.96464646464646464

### Example 5: Drawing on additional resources

One of the important ways to improve results on NLP problems is to get access to additional resources.  You always have a limited amount of training data for the problem you're interested in, which will include instances with properties that you have seen rarely if at all.  If you have some general sense of how words behave, from text corpora or dictionaries, you can use that information to generalize from words that you have seen to words that you haven't.

Word embeddings, of course, are a prototypical example of this.  To use them, you just add features to your data points computed from the word embeddings.  The sparse matrix `stack` operations make it easy to construct the matrices you need. (Note: if you have extra features that aren't in the word embedding, e.g., bigrams, then you need to account for that in applying the generic word embeddings; conversely, you probably want to add the word embedding features to your problem, since you may actually have information about particular words that gives special but reliable information about your classification problem that you do not want to discard.)  With these hints, you can probably understand the code for adding embedding information to your classification problem.

Another thing you could try along these lines would be to build an embedding matrix by applying SVD to the training data, as in the latent semantic analysis exercise we did two weeks ago.

```python
_, _, wrt = scipy.sparse.linalg.svds(expt.X_train, k=r)
W = np.transpose(wrt))
```

In [13]:
def stack_embeddings(embeddings) :
    def operation(X, y) :
        extra_features = X.shape[1] - embeddings.shape[0]
        if extra_features > 0 :
            Z = scipy.sparse.csr_matrix((extra_features, embeddings.shape[1]))
            W = scipy.sparse.vstack([embeddings, Z])
        else: 
            W = embeddings
        return scipy.sparse.hstack([X, X.dot(W)]), y
    return operation

expt3 = Experiment.transform(expt2,
                             stack_embeddings(e),
                            sklearn.linear_model.SGDClassifier(loss="log",
                                       penalty="elasticnet",
                                       n_iter=5))
expt3.fit_and_validate()
expt3.accuracy

0.96969696969696972

### Example 6: Fancy feature definitions

The place that you get a payoff from a good design is when you're able to easily try something out that you didn't anticipate when you started writing the code.  Invariably, that's a bit lucky when it works out, and anything that you start pushing too hard in a direction that it wasn't designed for becomes more and more cumbersome until you can't stand it.  But we're not there yet in this assignment.

Last time we looked at [nltk's resources for analyzing collocations][1], because, as we saw, it's an interesting unsupervised technique for characterizing word senses and dealing with ambiguous words.  The architecture that we have for identifying features in the training data makes it possible to preprocess the training data to look for meaningful bigram features.  (The strategy I've used here is not exactly the one that [Wang and Manning][2] recommend by the way!)  Python generator functions make it straightforward to report instances of these features in the course of scanning through a file.  

So it's totally routine to try this out by adapting the work we've done so far.  Who knows what else we could handle this way!

[1]:http://www.nltk.org/howto/collocations.html
[2]:http://aclweb.org/anthology/P/P12/P12-2018.pdf

In [14]:
def use_bigram_features(data) :
    f = vocabulary.Vocabulary.load("vocab.pkl", file_type="pickle")
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    word_fd = nltk.FreqDist(data.all_train_tokens())
    bigram_fd = nltk.FreqDist(nltk.bigrams(data.all_train_tokens()))
    finder = nltk.collocations.BigramCollocationFinder(word_fd, bigram_fd)
    finder.apply_freq_filter(5)
    collocations = filter(lambda (g,i): i > 0, finder.score_ngrams(bigram_measures.pmi))
    for (w1, w2), _ in collocations:
        f.add(w1 + " " + w2)
    f.stop_growth()
    return f

def count_bigram_features(features, gen_tokens) :
    prev = None
    for t in gen_tokens :
        r = features.add(t)
        if r :
            yield r
            if prev :
                r = features.add(prev + " " + t)
                if r : 
                    yield r
            prev = t

In [15]:
autos_vs_religion_b = newsreader.DataManager(train_dir + AUTOS,
                                             train_dir + RELIGION,
                                             test_dir + AUTOS,
                                             test_dir + RELIGION,
                                             use_bigram_features,
                                             make_binary_features(count_bigram_features))
autos_vs_religion_b.initialize()

In [16]:
expt4 = Experiment(autos_vs_religion_b,
                   sklearn.linear_model.SGDClassifier(loss="log",
                                       penalty="elasticnet",
                                       n_iter=5))
expt4.initialize()
expt4.fit_and_validate()

expt4.accuracy

0.92929292929292928

In [17]:
expt5 = Experiment.transform(expt4,
                             stack_embeddings(e),
                            sklearn.linear_model.SGDClassifier(loss="log",
                                       penalty="elasticnet",
                                       n_iter=5))
expt5.fit_and_validate()

expt5.accuracy

0.94444444444444442